In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df = pd.read_csv("assets.csv")
df = df[df["type"] == "model"]
#df.drop(columns=["sample", "datasheet", "included", "excluded", "adaptation", "output_space", "terms_of_service", "monthly_active_users", "user_distribution", "failures"], inplace=True)
df["created_date"] = pd.to_datetime(df["created_date"], format="%d/%m/%Y")
df["inputs"] = df.modality.str.split("; ").str[0].str.split(", ")
df["outputs"] = df.modality.str.split("; ").str[1].str.split(", ")
df.drop(columns="modality", inplace = True)
df["size"] = df["size"].apply(lambda x: np.nan if x == "unknown" else x)
df["dense"] = df["size"].str.endswith("(dense)")
df["dense"].fillna(False, inplace=True)
df["parameters"] = df["size"].str.extract(r"^(\d+[a-zA-Z])")
df["parameters_(millions)"] = df["parameters"].str.replace("M", "").str.replace("B", "000").str.replace("T", "000000")
df["parameters_(millions)"] = pd.to_numeric(df["parameters_(millions)"], downcast="integer")
df.drop(columns=["size", "parameters"], inplace=True)
df["dependencies"] = df["dependencies"].apply(lambda x: x[1:-1].split(','))
df["analysis"].fillna("unknown", inplace=True)
df["access"] = df["access"].map({"open":2, "limited":1, "closed": 0})
df["training_time"].fillna("unknown", inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 359 entries, 3 to 564
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   type                   359 non-null    object        
 1   name                   359 non-null    object        
 2   organization           359 non-null    object        
 3   description            299 non-null    object        
 4   created_date           357 non-null    datetime64[ns]
 5   url                    357 non-null    object        
 6   datasheet              0 non-null      object        
 7   sample                 0 non-null      object        
 8   analysis               359 non-null    object        
 9   dependencies           359 non-null    object        
 10  included               0 non-null      object        
 11  excluded               0 non-null      object        
 12  quality_control        85 non-null     object        
 13  access    

In [28]:
pd.options.display.max_columns = None

In [22]:
df[df["name"].str.contains("Grok")]

,type,name,organization,description,created_date,url,datasheet,sample,analysis,dependencies,...,adaptation,output_space,terms_of_service,monthly_active_users,user_distribution,failures,inputs,outputs,dense,parameters_(millions)
507,model,Grok-1,xAI,Grok is an AI modeled after the Hitchhiker’s G...,2023-11-04,https://grok.x.ai/,NaN,NaN,Grok-1 was evaluated on a range of reasoning b...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],True,314000.0
508,model,Grok-1.5V,xAI,Grok-1.5V is a first-generation multimodal mod...,2024-04-12,https://x.ai/blog/grok-1.5v,NaN,NaN,The model is evaluated in a zero-shot setting ...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,"[image, text]",[text],False,NaN


In [26]:
shortlist = df.loc[[268, 212, 399, 428, 412, 56, 65, 445, 427, 508]]

In [27]:
shortlist.columns

Index(['type', 'name', 'organization', 'description', 'created_date', 'url',
       'datasheet', 'sample', 'analysis', 'dependencies', 'included',
       'excluded', 'quality_control', 'access', 'license', 'intended_uses',
       'prohibited_uses', 'monitoring', 'feedback', 'model_card',
       'training_emissions', 'training_time', 'training_hardware',
       'adaptation', 'output_space', 'terms_of_service',
       'monthly_active_users', 'user_distribution', 'failures', 'inputs',
       'outputs', 'dense', 'parameters_(millions)'],
      dtype='object')

In [40]:
shortlist["name"]

268            GPT-4
212           Gemini
399          Llama 3
428           Vicuna
412         Claude 3
56        StableLM 2
65     Mistral Large
445      Falcon-180B
427             DBRX
508        Grok-1.5V
Name: name, dtype: object